# Equal Weighted Like Portfolio - comparisons

This example compares the performance of an equal weighted portfolio relative to several risk-based optimal portfolios with similar risk profile. 

The CVaR, SMCR, EVaR, MAD, LSD, BTAD, BTSD, GINI, SD and MV risk-based portfolios are considered. We use the `rtype=InvNrisk` optimization flag (the optimal portfolio with the same risk as the equal weights portfolio).

The results are presented below. We concluded that, at least for the data in this example, there is no clear-cut preference for a portfolio construction (with a superior performance under all market conditions).


We start by importing **azapy** and other useful packages.

In [57]:
import sys
sys.path.append("..")
import azapy as az

print(f"azapy version {az.version()}")

azapy version 1.2.2


### Collect historical market data

- `symb` is the list of stock symbols (portfolio components).
- `sdate` and `edate` are the start and end dates of historical time-series.
- `mktdir` is the name of the directory used as a buffer for market data collected from the data provider (in this case _alphavantage_).
    
> Note: if the flag `force=False` then a reading from `dir=mktdir` is attempted. If it fails, then the data provider servers will be accessed. The new data will be saved to the `dir=mktdir`. For more information see the readMkT documentation https://azapy.readthedocs.io/en/latest/.

In [58]:
symb = ['GLD', 'TLT', 'XLV', 'VGT', 'VHT']

sdate = "2012-01-01"
edate = "2021-07-27"
mktdir = "../MkTdata"

mktdata = az.readMkT(symb, sdate=sdate, edate=edate, file_dir=mktdir)

read GLD data from file
read TLT data from file
read XLV data from file
read VGT data from file
read VHT data from file

Request between 2012-01-01 : 2021-07-27
                    GLD         TLT         XLV         VGT         VHT
source            yahoo       yahoo       yahoo       yahoo       yahoo
force             False       False       False       False       False
save               True        True        True        True        True
file_dir     ../MkTdata  ../MkTdata  ../MkTdata  ../MkTdata  ../MkTdata
file_format         csv         csv         csv         csv         csv
api_key            None        None        None        None        None
verbose            True        True        True        True        True
error                No          No          No          No          No
nrow               2407        2407        2407        2407        2407
sdate        2012-01-03  2012-01-03  2012-01-03  2012-01-03  2012-01-03
edate        2021-07-27  2021-07-27  2021-07-27

### Set scenarios model parameters

Since we plan to run many scenarios, it is useful to organize their model parameters in a dictionary.
The keys are the portfolio names, and the values are the model parameters organized also as dictionaries:
- 'type' : is the portfolio class used in the simulation,
- 'm_param' : is a dictionary with parameters for `set_model` functions.

Note that:
- `'P_N'` is the reference equal weighted portfolio (EWP),
- `'P_CVaR'` is the mCVaR optimal portfolio with confidence levels `alpha_CVaR` equal weighted,
- `'P_SMCR'` is the mSMCR optimal portfolio with confidence levels `alpha_SMCR` equal weighted,
- `'P_EVaR'` is the mEVaR optimal portfolio with confidence levels `alpha_EVaR` equal weighted,
- `'P_MAD'` is the mMAD optimal portfolio with weights `coef_MAD`,
- `'P_LSD'` is the mLSD optimal portfolio with weights `coef_LSD`,
- `'P_BTAD'` is the mBTAD optimal portfolio with threshold levels `alpha_BTAD` equal weighted,
- `'P_BTSD'` is the mBTSD optimal portfolio with threshold levels `alpha_BTSD` equal weighted,
- `'P_MV'` is the MV (mean-variance) optimal portfolio,
- `'P_SD'` is the SD optimal portfolio.
- `'P_Gini'` is the Gini optimal portfolio. It is commented out in the `models` dictionary since it is very slow.
- In all cases the optimization type is set to `rtype='InvNRisk'` (optimal portfolio with the same risk profile as the equal weighted portfolio)
- The `'hlength'` parameter (the length of historical data used in weights calibration) is set to 3.25 years except for `'P_GINI'` where it is set to 1.25 years (for computational speed convenience).

> In this case P_MV and P_SD are the same. 

In [59]:
rtype = 'InvNrisk'
alpha_CVaR = [0.95, 0.90, 0.85]
alpha_SMCR = [0.9, 0.80]
alpha_EVaR = [0.75, 0.65]
coef_MAD = [1./3.] * 3
coef_LSD = [1./3.] * 3
alpha_BTAD = [-0.01, 0]
alpha_BTSD = [-0.01, 0]
hlength = 1.25
verbose = False

models = {'P_CVaR': {'type': 'Port_CVaR', 'm_param': {'alpha': alpha_CVaR, 'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_SMCR': {'type': 'Port_SMCR', 'm_param': {'alpha': alpha_SMCR, 'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_EVaR': {'type': 'Port_EVaR', 'm_param': {'alpha': alpha_EVaR, 'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_MAD': {'type': 'Port_MAD', 'm_param': {'coef': coef_MAD, 'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_LSD': {'type': 'Port_LSD', 'm_param': {'coef': coef_LSD, 'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_BTAD': {'type': 'Port_BTAD', 'm_param': {'alpha': alpha_BTAD, 'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_BTSD': {'type': 'Port_BTSD', 'm_param': {'alpha': alpha_BTSD, 'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          #'P_GINI': {'type': 'Port_GINI', 'm_param': {'rtype': rtype, 'hlength': 1.25, 'verbose': verbose}},
          'P_MV': {'type': 'Port_MV', 'm_param': {'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_SD': {'type': 'Port_SD', 'm_param': {'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_N': {'type': 'Port_ConstW', 'm_param': {'ww': None}}}

### Main computation loop

- `port` is a list containing the simulated time-series. We will use it to setup a simple portfolio *(see the documentation for `Port_Simple` class)*. It is a very convenient way to facilitate the visual and numerical comparisons between these portfolio performances. 
- `pp` is a dictionary holding the portfolio objects. They may be used later for further analytical inquires.  

In [60]:
port = []
pp = {}
for key, val in models.items():
    ppz = getattr(az, val['type'])
    pp_ = ppz(mktdata, pname=key)
    pp[key] = pp_
    port_ = pp_.set_model(**val['m_param'])
    port.append(port_)

### Prepare the results for comparisons 

Build a `Port_Simple` class holding all the computed portfolios as components. We use this structure to build comparative graphical and numerical performance reports. The aggregated portfolio of portfolios time-series will be neglected.

>Note the call to `set_model` method that is a must.

>Observation: `Port_Simple` is the class that supports the back testing of "Buy and Hold" portfolio (_see its documentation_).
It also can be used as a tool to compare the performance of multiple portfolios. Here we use it in this latter capacity.

In [61]:
ps = az.Port_Simple(port, col='close', pname='ALL')
_ = ps.set_model()

### Time-series visualization 

We used the flag `componly=True` to plot only the portfolio components. 

In [62]:
_ = ps.port_view_all(componly=True, fancy=True, title="Relative performance")

### Portfolio performances

Note that, at least for the duration of this simulation, the P_BTAD portfolio has a similar rate of returns as 
P_N. However, P_BTAD has higher maximum drawdown than P_N. P_N has the smallest maximum drawdown among all portfolios. 

> Note: in this case P_MV anf P_SD portfolios are the same.

In [63]:
ps.port_perf(componly=True, fancy=True)

,RR,DD,RoMaD,DD_date,DD_start,DD_end
symbol,,,,,,
P_CVaR,14.23,-15.27,0.932072,2016-12-14,2016-07-29,2017-05-25
P_LSD,13.97,-15.32,0.911515,2016-12-14,2016-09-29,2017-05-25
P_EVaR,13.69,-15.31,0.894414,2016-12-14,2016-08-23,2017-05-25
P_SMCR,13.09,-14.75,0.887566,2020-03-18,2020-03-06,2020-03-30
P_N,13.32,-15.92,0.836395,2020-03-20,2020-02-19,2020-04-09
P_BTSD,13.64,-17.59,0.775415,2016-12-14,2015-07-16,2017-09-11
P_MV,13.35,-17.68,0.755064,2016-12-14,2015-07-16,2017-09-19
P_SD,13.35,-17.68,0.755064,2016-12-14,2015-07-16,2017-09-19
P_BTAD,12.78,-19.47,0.656515,2016-12-14,2015-07-20,2017-09-29


### Annual returns

Remarks:

- 2015 (only half year in this simulation) and 2016, P_N is the best performer.
- 2016 P_N is the best while P_GINI is the worst.
- 2017 P_N is the worst performer. The best performers are P_GINI.
- 2018 P_GINI has the best performance. P_N is second.
- 2019 The best performers are P_BTAD closely follow by P_CVaR, P_MAD and P_LSD. P_GINI and P_N are the worst.
- 2020 The best performers are P_GINI follow by P_N.
- 2021 (first half of the year in this simulation) P_GINI is the leader followed by P_N and P_MV (P_SD is the same with P_MV).

Conclusion: Depending on the historical period P_N may be an outperformer or an underperformer. Moreover, similar remarks can be made about any other portfolio strategy.

> Note: the flags `withcomp=True` includes the portfolios components while the flag `componly=True` excludes the aggregated portfolio of portfolios.


In [64]:
ps.port_annual_returns(withcomp=True, componly=True, fancy=True)

symbol,P_BTAD,P_BTSD,P_CVaR,P_EVaR,P_LSD,P_MAD,P_MV,P_N,P_SD,P_SMCR
year,,,,,,,,,,
2015,-10.19%,-9.57%,-5.68%,-6.75%,-7.19%,-11.01%,-9.45%,-3.17%,-9.45%,-3.78%
2016,-7.76%,-6.39%,-6.58%,-6.14%,-6.11%,-7.48%,-6.85%,5.33%,-6.85%,-6.28%
2017,33.43%,32.56%,31.54%,31.82%,32.12%,33.59%,32.26%,19.26%,32.26%,30.87%
2018,5.39%,7.70%,6.99%,6.81%,6.95%,6.73%,7.81%,4.99%,7.81%,6.15%
2019,16.73%,16.38%,16.06%,14.99%,15.54%,16.62%,17.34%,22.80%,17.34%,12.59%
2020,41.14%,42.51%,42.96%,41.49%,42.73%,40.83%,40.50%,25.29%,40.50%,39.04%
2021,7.25%,7.56%,8.25%,7.93%,8.09%,7.11%,7.13%,7.48%,7.13%,6.98%


### Monthly returns

> Note: the flags `withcomp=True` includes the portfolios components while the flag `componly=True` excludes the aggregated portfolio. 

In [65]:
ps.port_monthly_returns(withcomp=True, componly=True, fancy=True)

### A closer look at P_BTAD portfolio

Monthly returns: are volatile.

In [66]:
pp['P_BTAD'].port_monthly_returns(fancy=True)

year,2015,2016,2017,2018,2019,2020,2021
month,,,,,,,
1,nan%,-1.69%,3.88%,5.83%,5.75%,4.75%,-1.10%
2,nan%,1.32%,3.95%,-1.58%,1.89%,2.01%,-3.15%
3,nan%,3.09%,0.80%,1.02%,-1.47%,3.43%,-2.51%
4,nan%,-2.37%,2.64%,1.21%,-1.82%,10.79%,2.32%
5,nan%,2.86%,3.48%,2.52%,1.62%,5.36%,-0.36%
6,-1.89%,3.44%,-5.57%,-2.71%,3.76%,1.68%,4.97%
7,2.01%,1.50%,4.98%,0.43%,-0.24%,4.89%,3.11%
8,-7.72%,0.30%,2.56%,4.74%,3.96%,-1.26%,nan%
9,-6.66%,-2.02%,2.86%,-0.04%,-3.08%,-0.06%,nan%


### P_BTAD performance

An example of individual portfolio performance inquiry. 

In [67]:
pp['P_BTAD'].port_perf(fancy=True)

,RR,DD,RoMaD,DD_date,DD_start,DD_end
symbol,,,,,,
P_BTAD,12.78,-19.47,0.656515,2016-12-14,2015-07-20,2017-09-29
VGT,23.10,-31.84,0.725512,2020-03-23,2020-02-19,2020-06-09
VHT,17.41,-28.85,0.603515,2020-03-23,2020-02-19,2020-06-08
XLV,16.74,-28.40,0.589180,2020-03-23,2020-01-22,2020-07-15
TLT,4.89,-21.34,0.229103,2021-03-18,2020-08-04,NaN
GLD,0.81,-42.11,0.019280,2015-12-17,2012-10-04,2020-07-22


### P_BTAD drawdowns

Remarks: The first 2 largest drawdowns are similar. However, the 2018 drawdown was caused by the unexpected Feds rate increase while the 2020 event is the Covid-19 pandemic crash. 

In [68]:
pp['P_BTAD'].port_drawdown(fancy=True)

,DD,Date,Start,End
No,,,,
1,-19.47,2016-12-14,2015-07-20,2017-09-29
2,-14.35,2020-03-18,2020-03-09,2020-03-30
3,-11.97,2018-12-24,2018-08-31,2019-01-31
4,-10.79,2018-02-08,2018-01-26,2018-06-06
5,-9.11,2021-03-08,2021-02-11,2021-06-17


### P_BTAD weights

Remarks: At present the P_BTAD portfolio is dominated by GLD and VGT.

In [69]:
pp['P_BTAD'].get_weights(fancy=True)

,Droll,Dfix,GLD,TLT,XLV,VGT,VHT
0,2015-06-25,2015-06-24,2.33,0.00,0.00,0.00,97.67
1,2015-09-25,2015-09-24,23.26,8.09,0.00,4.40,64.25
2,2015-12-28,2015-12-24,10.29,24.93,0.00,48.33,16.45
3,2016-03-28,2016-03-24,0.00,54.77,8.71,36.51,0.00
4,2016-06-27,2016-06-24,0.00,88.21,0.00,11.79,0.00
5,2016-09-27,2016-09-26,12.30,87.52,0.00,0.18,0.00
6,2016-12-27,2016-12-23,33.94,0.00,0.00,66.06,0.00
7,2017-03-28,2017-03-27,0.00,0.00,0.00,100.00,0.00
8,2017-06-27,2017-06-26,0.00,0.00,0.00,100.00,0.00
9,2017-09-26,2017-09-25,0.00,0.00,0.00,100.00,0.00
